In [3]:
import pygame as py
import sys
import numpy as np
from pygame.locals import *
from keras.models import load_model
import cv2


In [4]:
#initialize our pygame
py.init()
FONT=py.font.Font("FreeSansBold.ttf",18)
WINX=840
WINY=680

WHITE=(255,255,255)
BLACK=(0,0,0)
RED=(255,0,0)

IMAGESAVE=False

iswriting=False

boundary=5

img_cnt=1

PREDICT=True

n_xcord=[]
n_ycord=[]

MODEL=load_model("MCDC2.h5")

LABELS={0:"ZERO",1:"ONE",2:"TWO",3:"THREE",4:"FOUR",5:"FIVE",6:"SIX",7:"SEVEN",8:"EIGHT",9:"NINE"}

DisSur=py.display.set_mode((WINX,WINY))

py.display.set_caption("Digit Board")

In [5]:
while True:
    for event in py.event.get():
        if event.type==QUIT:
            py.quit()
            sys.exit()
            
        if event.type == MOUSEMOTION and iswriting:
            xcord,ycord = event.pos
            py.draw.circle(DisSur,WHITE,(xcord,ycord),4,0)
            n_xcord.append(xcord)
            n_ycord.append(ycord)
            
        if event.type == MOUSEBUTTONDOWN:
            iswriting = True
        
        if event.type == MOUSEBUTTONUP:
            iswriting=False
            if n_xcord and n_ycord:
                n_xcord=sorted(n_xcord)
                n_ycord=sorted(n_ycord)
                rect_min_x = max(n_xcord[0] - boundary, 0)
                rect_min_y = max(n_ycord[0] - boundary, 0)
                rect_max_x = min(n_xcord[-1] + boundary, WINX - 1)  # Adjusted to avoid index out of bounds
                rect_max_y = min(n_ycord[-1] + boundary, WINY - 1) 
                n_xcord=[]
                n_ycord=[]
            
                img_arr=np.array(py.PixelArray(DisSur))[rect_min_x:rect_max_x,rect_min_y:rect_max_y].T.astype(np.float32)
            
            if IMAGESAVE:
                cv2.imwrite("image.png", img_arr) 
                image_cnt+=1
                
            if PREDICT:
                
                image=cv2.resize(img_arr,(28,28))
                image=np.pad(image,(10,10),'constant',constant_values=0)
                image=cv2.resize(image,(28,28))/255
                
                label=str(LABELS[np.argmax(MODEL.predict(image.reshape(1,784)))])
                
                textsurface = FONT.render(label,True,RED,WHITE)
                textRecobj=textsurface.get_rect()
                textRecobj.left,textRecobj.bottom=rect_min_x,rect_max_y
                
                DisSur.blit(textsurface,textRecobj)
                
        if event.type == KEYDOWN:
            if event.unicode=="n":
                DisSur.fill(BLACK)
    
    py.display.update()
         
            

1/1 [==============================] - 0s 31ms/step


SystemExit: 

C:\Users\Admin\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
